<a href="https://colab.research.google.com/github/nipuni1313/CNN-for-image-classification/blob/main/2_SOTA_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Create train, val, and test directories
train_dir = '/content/drive/MyDrive/RealWasteDataset_D/train/'
val_dir = '/content/drive/MyDrive/RealWasteDataset_D/validation/'
test_dir = '/content/drive/MyDrive/RealWasteDataset_D/test/'
import os
import shutil
from sklearn.model_selection import train_test_split

# Path to your image dataset
dataset_path = '/content/drive/MyDrive/RealWaste_D/'

# List all categories (assumes subfolders for each category)
categories = os.listdir(dataset_path)



for dir in [train_dir, val_dir, test_dir]:
    if not os.path.exists(dir):
        os.makedirs(dir)

# Create directories for each category inside train, val, and test
for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(val_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)


In [3]:
# Split the dataset into train, validation, and test sets
for category in categories:
    category_path = os.path.join('/content/drive/MyDrive/RealWaste_D/', category)

    # Check if the category path exists and is a directory
    if os.path.isdir(category_path) and len(os.listdir(category_path)) > 0:
        images = os.listdir(category_path)

        # Split the data into 60% train, 20% validation, 20% test
        train_images, temp_images = train_test_split(images, test_size=0.4, random_state=42)
        val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

        # Create target category directories in train, val, and test directories if they don't exist
        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(val_dir, category), exist_ok=True)
        os.makedirs(os.path.join(test_dir, category), exist_ok=True)

        # Move files to respective directories
        for image in train_images:
            # Check if the image is already in the target directory
            target_path = os.path.join(train_dir, category, image)
            if not os.path.exists(target_path):  # Only move if the image doesn't exist
                shutil.move(os.path.join(category_path, image), target_path)

        for image in val_images:
            # Check if the image is already in the target directory
            target_path = os.path.join(val_dir, category, image)
            if not os.path.exists(target_path):  # Only move if the image doesn't exist
                shutil.move(os.path.join(category_path, image), target_path)

        for image in test_images:
            # Check if the image is already in the target directory
            target_path = os.path.join(test_dir, category, image)
            if not os.path.exists(target_path):  # Only move if the image doesn't exist
                shutil.move(os.path.join(category_path, image), target_path)

print("Dataset split into train, validation, and test directories.")

Dataset split into train, validation, and test directories.


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize ImageDataGenerator for data augmentation (train only)
train_datagen = ImageDataGenerator(
    rescale=1./255,        # Normalize pixel values to [0, 1]
    rotation_range=20,     # Random rotations
    width_shift_range=0.2, # Random horizontal shift
    height_shift_range=0.2, # Random vertical shift
    shear_range=0.2,       # Shear transformation
    zoom_range=0.2,        # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'    # Fill empty pixels
)

# For validation and test, we just rescale the images
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data using flow_from_directory
train_data = train_datagen.flow_from_directory(
    train_dir,  # Directory with training data
    target_size=(256, 256),    # Resize images
    batch_size=32,
    class_mode='sparse'  # Set to 'sparse' to get integer labels
)

val_data = val_test_datagen.flow_from_directory(
    val_dir,  # Directory with validation data
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse'  # Set to 'sparse' to get integer labels
)

test_data = val_test_datagen.flow_from_directory(
    test_dir,  # Directory with test data
    target_size=(256, 256),
    batch_size=32,
    class_mode='sparse'  # Set to 'sparse' to get integer labels
)


Found 2902 images belonging to 9 classes.
Found 968 images belonging to 9 classes.
Found 972 images belonging to 9 classes.


In [5]:
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

### ResNet50

In [6]:
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers in the base model
for layer in resnet_base.layers:
    layer.trainable = False

# Add custom layers on top
resnet_model = Sequential([
    resnet_base,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_data.class_indices), activation='softmax')  # Output layer
])

resnet_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_resnet = resnet_model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,  # Set the number of epochs as needed
    steps_per_epoch=len(train_data),
    validation_steps=len(val_data)
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.1202 - loss: 2.6684 

In [ ]:
resnet_test_loss, resnet_test_acc = resnet_model.evaluate(test_data, steps=len(test_data))
print(f"ResNet50 Test Accuracy: {resnet_test_acc}")

### VGG16

In [ ]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the layers in the base model
for layer in vgg_base.layers:
    layer.trainable = False

# Add custom layers on top
vgg_model = Sequential([
    vgg_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_data.class_indices), activation='softmax')  # Output layer
])

vgg_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_vgg = vgg_model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,  # Set the number of epochs as needed
    steps_per_epoch=len(train_data),
    validation_steps=len(val_data)
)


In [ ]:
vgg_test_loss, vgg_test_acc = vgg_model.evaluate(test_data, steps=len(test_data))
print(f"VGG16 Test Accuracy: {vgg_test_acc}")


Visualizing the training results

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history, model_name):
    plt.figure(figsize=(12, 4))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_history(history_resnet, "ResNet50")
plot_history(history_vgg, "VGG16")
